In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import Tool
from typing import TypedDict
from dotenv import load_dotenv
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode, tools_condition

import gradio as gr

In [ ]:
load_dotenv(override=True)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
model_name = "gemini-2.5-flash-lite"
model = ChatGoogleGenerativeAI(
    model=model_name,
    max_tokens=None,
    timeout=None
)

In [ ]:
def get_recs_team_members(role: str):
    if role == "Developer":
        return ["Jon Snow", "Cersei Lannister", "Tyrion Lannister", "Daenerys Targaryen"]
    elif role == "Product Owner":
        return ["Sansa Stark"]
    elif role == "tio" or role == "devops":
        return ["Brienne of Tarth"]
    else:
        return []

recs_members = Tool(
    func=get_recs_team_members,
    name="get_recs_team_members",
    description="Useful for checking Recommenders team members. Input should be the role (e.g., 'Developer', 'Product Owner', 'devops'). The output is a dictionary of names (e.g., ['Jon Snow', 'Cersei Lannister']) and may require further processing"
)

In [ ]:
tools = [recs_members]
model_with_tools = model.bind_tools(tools)

In [ ]:
def say_hello(old_state: State) -> State:
    response = model_with_tools.invoke(old_state["messages"])
    print(response)
    return State(messages=response)



In [ ]:

graph_builder.nodes.clear()
graph_builder.add_node("chatbot", say_hello)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
graph_builder.edges.clear()
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")

In [ ]:
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
recs_members.invoke("Developer")

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    state = State(messages=[user_input])
    result = graph.invoke(state, config=config)
    return result['messages'][-1].content


gr.ChatInterface(chat).launch()